In [ ]:
!apt-get update && apt-get install wget -y

In [1]:
#!apt-get update && apt-get install libopenexr-dev -y
!python3 -m pip install mediapy einshape chex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 854.7 kB/s eta 0:00:001m1.4 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.4 MB/s eta 0:00:000m eta 0:00:01:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.6 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/deepmind/tapnet.git

Cloning into 'tapnet'...
remote: Enumerating objects: 787, done.
remote: Counting objects: 100% (460/460), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 787 (delta 283), reused 347 (delta 244), pack-reused 327
Receiving objects: 100% (787/787), 1.40 MiB | 2.40 MiB/s, done.
Resolving deltas: 100% (473/473), done.


In [4]:


!cd tapnet/tapnet && pip3 install -r requirements.txt

  Using cached jupyter_http_over_ws-0.0.8-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached jaxline-0.0.8-py3-none-any.whl
  Using cached optax-0.2.2-py3-none-any.whl.metadata (8.1 kB)
  Using cached dm_haiku-0.0.12-py3-none-any.whl.metadata (19 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (1.9 kB)
  Using cached tensorflow-2.16.1-cp39-cp39-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (4.1 kB)
  Using cached tensorflow_datasets-4.9.3-py3-none-any.whl.metadata (9.3 kB)
  Using cached tensorflow_graphics-2021.12.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached ml_collections-0.1.1-py3-none-any.whl
  Using cached wrapt-1.16.0-cp39-cp39-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (6.6 kB)
  Using cached jmp-0.0.4-py3-none-any.whl.metadata (8.9 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached flax-0.8.3-py3-none-any.whl.metadata (10 kB)
  Using cached astunparse-1.6.3-

In [28]:
# !export PYTHONPATH=`(cd ./ && pwd)`:`pwd`:$PYTHONPATH

In [1]:
from multiprocessing import Process
from multiprocessing import Event as PEvent
from multiprocessing import Queue as PQueue
import cv2 
from time import sleep, time
class Camera():

    def __init__(self, cap_id):
        self._cap = cv2.VideoCapture(cap_id)
        assert self._cap.isOpened(), f'can not connect to camera with id {cap_id}'
        

    def run(self, stop_event, outque):
        time_out = time()
        print('start polling frames')
        while not stop_event.is_set():
            try:
                ret, frame = self._cap.read()
                outque.put([frame, time()])
                #print(time() - time_out, end = '        \r')
                if outque.full():
                    frame_out, time_out = outque.get()
            except KeyboardInterrupt:
                pass
             
        self._cap.release()
        print()
        print(f'frames polling stopped               {not self._cap.isOpened()}',)
vidstop = PEvent()
vidque = PQueue(2)
vcap = Camera(0)
video_poll_process = Process(target=vcap.run, args=(vidstop, vidque, ))
video_poll_process.start()

start polling frames


In [5]:
# vcap._cap.release()

In [4]:
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
import torchvision.transforms as T

from tapnet.torch import tapir_model
from tapnet.utils import transforms
from tapnet.utils import viz_utils

import cv2 
import numpy as np
from time import sleep, time

from IPython.display import clear_output

def postprocess_occlusions(occlusions, expected_dist):
  visibles = (1 - F.sigmoid(occlusions)) * (1 - F.sigmoid(expected_dist)) > 0.5
  return visibles


def preprocess_frames(frames):
  """Preprocess frames to model inputs.

  Args:
    frames: [num_frames, height, width, 3], [0, 255], np.uint8

  Returns:
    frames: [num_frames, height, width, 3], [-1, 1], np.float32
  """
  frames = frames.float()
  frames = frames / 255 * 2 - 1
  return frames


def transform(frame):
    frame =  cv2.resize(frame, (fsize , fsize))
    tframe = torch.tensor(frame).permute(2,0,1)
    crop = tframe.float().permute(1,2,0)
    # crop = T.CenterCrop(fsize)(tframe).float().permute(1,2,0)
    return crop / 255 * 2 - 1
    


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device = torch.device('cpu')
model = tapir_model.TAPIR(pyramid_level=1, initial_resolution = (64, 64))

# model.load_state_dict(torch.load('tapnet/tapir_checkpoint_panning.pt'))
model.load_state_dict(torch.load('tapnet/bootstapir_checkpoint.pt'))
model = model.to(device)

In [3]:
fsize = 64#256
step = 16#64*4

points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)

tcrop = None
from IPython.display import display as IPdisplay, Image as IPImage

display_handle=IPdisplay(None, display_id=True)

try:
    while True:
        tic = time()
        # ret, frame = cap.read()
        frame, _ = vidque.get()
        
        crop = transform(frame)
        
        if tcrop is None:
            tframe = frame
            tcrop = crop
    
        seq = torch.cat((tcrop[None], crop[None]))[None]
    
        outputs = model(seq, points)
        tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]
    
        crops = []
        for img in [tcrop, crop]:
            img = ((img.numpy() + 1) * 255 / 2).astype(np.uint8)
            crops.append(img)
    
    
        for pts in tracks:
            color = np.random.randint(low=0, high=255,size=3).tolist()
            for pt, img in zip(pts, crops):
                cv2.circle(img, pt.numpy().astype(np.int32).tolist(), 1, color, -1)
    
        tcrop = crop
        
        tframe = frame

        for index_, crop in enumerate(crops):
            crop = cv2.resize(crop, (640, 480))
            crops[index_] = crop
        canvas = np.hstack(crops)
        
        # clear_output(wait=True)

        frame = np.ndarray.astype(canvas[...,::-1], np.float32)
        frame = cv2.putText(frame, f"{1/(time()-tic):.2f}", (50, 50) , cv2.FONT_HERSHEY_SIMPLEX ,  
                           1, (255, 0, 0), 2, cv2.LINE_AA)
        
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # display(Image.fromarray(canvas[...,::-1]))
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPImage(data=frame.tobytes()))
        
        # print(f'{1/(time()-tic):.2f}', end='          \r')
except KeyboardInterrupt:
    pass
finally:
    display_handle.update(None)
#Image.fromarray(canvas[...,::-1])

None

In [12]:
model

TAPIR(
  (resnet_torch): ResNet(
    (initial_conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
    (block_groups): ModuleList(
      (0): BlockGroup(
        (blocks): ModuleList(
          (0): BlockV2(
            (proj_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn_0): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (conv_0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (conv_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn_1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          )
          (1): BlockV2(
            (bn_0): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (conv_0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (conv_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pa

In [12]:
cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)

(array([[ 1.01116129e+00, -5.54411142e-02, -7.85273849e+01],
        [ 4.51192478e-02,  1.02322173e+00,  1.65777599e+01],
        [ 1.73663661e-05,  7.02974701e-05,  1.00000000e+00]]),
 array([[0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1]], dtype=uint8))

In [8]:
type(tracks)

torch.Tensor

In [ ]:
tracks.shape

In [ ]:
canvas = tframe.copy()



In [ ]:
seq = torch.cat((tcrop[None], crop[None]))[None]
seq.shape

In [ ]:
fsize = 256
tframes = torch.tensor(np.asarray(frames)).permute(0,3,1,2)
tcrops = T.CenterCrop(fsize)(tframes).float()
tcrops = preprocess_frames(tcrops)[None].to(device).permute(0,1,3,4,2)

step = 32
points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)
#points = points.float()[None]
tcrops.shape, points.shape 

In [ ]:
from time import time

num_frames, height, width = tcrops.shape[1:4]

# Model inference
tic = time()
outputs = model(tcrops, points)
print(num_frames/(time()-tic))

tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]

# Binarize occlusions
visibles = postprocess_occlusions(occlusions, expected_dist)

tracks = tracks.cpu().numpy()
visibles = visibles.cpu().numpy()

In [ ]:
#!wget https://storage.googleapis.com/dm-tapnet/tapir_checkpoint_panning.pt
#!wget https://storage.googleapis.com/dm-tapnet/bootstapir_checkpoint.pt

In [ ]:
import torchvision.transforms as t


dir(t)
#help(t.CenterCrop)